# Import Required Libraraies

In [1]:
import re
import numpy as np
import pandas as pd

# Read Data

In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_sub = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# Data Purification and Feature Engineering

## Name
### Extract titles from names

In [4]:
train_data['Title'] = train_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))
test_data['Title'] = test_data['Name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

# Display the unique titles
print(f'All unique titles in names (Train): {train_data["Title"].unique()}')
print(f'All unique titles in names (Test): {test_data["Title"].unique()}')

All unique titles in names (Train): ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']
All unique titles in names (Test): ['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr' 'Dona']


In [5]:
# Train Data
train_data['Title'] = train_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
train_data['Title'] = train_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')

# Test Data
test_data['Title'] = test_data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr','Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Other')
test_data['Title'] = test_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
test_data['Title'] = test_data['Title'].replace('Mme', 'Mrs')

In [6]:
print(f'Train Titles: {train_data["Title"].unique()}')
print(f'Test Titles: {test_data["Title"].unique()}')

Train Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']
Test Titles: ['Mr' 'Mrs' 'Miss' 'Master' 'Other']


### Drop Names

In [7]:
train_data = train_data.drop('Name', axis=1)
test_data = test_data.drop('Name', axis=1)

### Encode Titles

In [8]:
title_to_number = {'Mr': 0, 'Mrs': 1, 'Miss': 2, 'Master': 3, 'Other': 4}

train_data['Title'] = train_data['Title'].apply(lambda x: title_to_number[x])
test_data['Title'] = test_data['Title'].apply(lambda x: title_to_number[x])

## Ticket
### Drop ticket columns

In [9]:
# Drop Ticket Column
train_data = train_data.drop('Ticket', axis=1)
test_data = test_data.drop('Ticket', axis=1)

## Sex
### Convert sexes to 0s and 1s

In [10]:
# Encode Sex
train_data['Sex'] = train_data['Sex'].replace({'male': 0, 'female': 1})
test_data['Sex'] = test_data['Sex'].replace({'male': 0, 'female': 1})
print(f'Number of distinct values for sex: {set(test_data["Sex"])}')

Number of distinct values for sex: {0, 1}


## Cabin
### Extract letters from Cabins

In [11]:
train_data['Cabin'] = train_data['Cabin'].fillna("X")
print(train_data['Cabin'].str[0].unique())

test_data['Cabin'] = test_data['Cabin'].fillna("X")
print(test_data['Cabin'].str[0].unique())

['X' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']
['X' 'B' 'E' 'A' 'C' 'D' 'F' 'G']


### Encode letters

In [12]:
cabin_to_number = {'X': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'T': 8}

train_data['Cabin_Encoded'] = train_data['Cabin'].apply(lambda x: cabin_to_number[x[0]])
test_data['Cabin_Encoded'] = test_data['Cabin'].apply(lambda x: cabin_to_number[x[0]])

### Drop Cabin Column

In [13]:
train_data = train_data.drop('Cabin', axis=1)
test_data = test_data.drop('Cabin', axis=1)

## Embarked
### Fill Null values with 'S'

In [14]:
train_data['Embarked'].describe()

train_data['Embarked'] = train_data['Embarked'].fillna('S')
test_data['Embarked'] = test_data['Embarked'].fillna('S')

### Encode Embarked Values

In [15]:
embark_to_number = {'S': 0, 'C': 1, 'Q': 2}

train_data['Embarked'] = train_data['Embarked'].apply(lambda x: embark_to_number[x])
test_data['Embarked'] = test_data['Embarked'].apply(lambda x: embark_to_number[x])

## Age
### Fill null values of age with mean age of train set

In [16]:
mean_age = train_data['Age'].mean()

train_data['Age'] = train_data['Age'].fillna(mean_age)
test_data['Age'] = test_data['Age'].fillna(mean_age)

### Convert ages to int

In [17]:
train_data['Age'] = train_data['Age'].astype(int)
test_data['Age'] = test_data['Age'].astype(int)

## Fare

In [18]:
print(f'number of fare null values of train set: {train_data["Fare"].isna().sum()}')
print(f'number of fare null values of test set: {test_data["Fare"].isna().sum()}')

number of fare null values of train set: 0
number of fare null values of test set: 1


### Fill null values with mean fare

In [19]:
mean_fare = train_data['Fare'].mean()

train_data['Fare'] = train_data['Fare'].fillna(mean_fare)
test_data['Fare'] = test_data['Fare'].fillna(mean_fare)

### Convert fares to ints

In [20]:
train_data['Fare'] = train_data['Fare'].astype(int)
test_data['Fare'] = test_data['Fare'].astype(int)

## PassengerId (Train)
### Drop column

In [21]:
train_data = train_data.drop(['PassengerId'], axis=1)

### Getting data ready

In [22]:
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()
testPassengerIds = test_data['PassengerId']

# Data Normalization

In [23]:
for col in X_train.columns:
    train_column = X_train[col]
    test_column = X_test[col]
    
    mean_val = train_column.mean()
    std_val = train_column.std()
    
    X_train[col] = (train_column - mean_val) / std_val
    X_test[col] = (test_column - mean_val) / std_val

In [24]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_Encoded
0,0.826913,-0.737281,-0.579719,0.432550,-0.473408,-0.498667,-0.568518,-0.738132,-0.488186
1,-1.565228,1.354813,0.649747,0.432550,-0.473408,0.788962,1.004617,0.190788,1.397540
2,0.826913,1.354813,-0.272352,-0.474279,-0.473408,-0.498667,-0.568518,1.119709,-0.488186
3,-1.565228,1.354813,0.419222,0.432550,-0.473408,0.426816,-0.568518,0.190788,1.397540
4,0.826913,-0.737281,0.419222,-0.474279,-0.473408,-0.478548,-0.568518,-0.738132,-0.488186


In [25]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_Encoded
0,0.826913,-0.737281,0.342381,-0.474279,-0.473408,-0.498667,2.577752,-0.738132,-0.488186
1,0.826913,1.354813,1.341322,0.432550,-0.473408,-0.498667,-0.568518,0.190788,-0.488186
2,-0.369158,-0.737281,2.493947,-0.474279,-0.473408,-0.458429,2.577752,-0.738132,-0.488186
3,0.826913,-0.737281,-0.195511,-0.474279,-0.473408,-0.478548,-0.568518,-0.738132,-0.488186
4,0.826913,1.354813,-0.579719,0.432550,0.767199,-0.398071,-0.568518,0.190788,-0.488186


In [26]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

## Separate cross validation set from train data

In [27]:
# Set the random seed for reproducibility
np.random.seed(42)

# Shuffle the data
shuffle_indices = np.random.permutation(len(X_train))
X_train_shuffled = X_train.values[shuffle_indices]
Y_train_shuffled = Y_train.values[shuffle_indices]

# Define the proportion of data to be allocated for cross-validation
val_split = 0.2

# Calculate the number of examples for the cross-validation set
num_val = int(val_split * len(X_train_shuffled))

# Split the data into training and cross-validation sets
X_train = X_train_shuffled[:-num_val]
Y_train = Y_train_shuffled[:-num_val]
X_val = X_train_shuffled[-num_val:]
Y_val = Y_train_shuffled[-num_val:]

# Logistic Regression

In [28]:
class LogisticRegression:
    def __init__(self, max_iterations=1000, learning_rate=0.01, decay=0.9, epsilon=1e-5):
        self.max_iterations = max_iterations
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.W = None
        self.b = None

        
    def sigmoid(self, z):
        a = 1 / (1 + np.exp(-z))
        return a

    
    def binary_cross_entropy(self, y_pred, y):
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)).mean()
        return loss

    
    def gradient_descent(self, X, y):
        m, n = X.shape

        dz = self.sigmoid(X.dot(self.W) + self.b) - y
        dw = np.dot(X.T, dz) / m
        db = np.sum(dz) / m

        return dw, db

    
    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.W = np.random.randn(n_features) * 0.01
        self.b = 0
        previous_loss = np.inf
        lr = self.learning_rate

        for _ in range(self.max_iterations):
            # Calculate Gradients
            dw, db = self.gradient_descent(X, y)

            # Update Parameters
            self.W -= lr * dw
            self.b -= lr * db

            # Check for Early Stop
            current_loss = self.binary_cross_entropy(self.sigmoid(X.dot(self.W) + self.b), y)
            if np.abs(current_loss - previous_loss) < self.epsilon:
                print(f'Stopping Train at Iteration No.{_}')
                break

            previous_loss = current_loss
            lr *= 0.92

        print(f'Training Complete with Binary Cross Entropy Loss of {current_loss}')

        
    def predict(self, X):
        y_pred = self.sigmoid(np.dot(X, self.W) + self.b)
        return (y_pred >= 0.5).astype(int)

## Check different hyperparameters

In [29]:
max_iterations_values = [100, 200, 500, 1000]
learning_rate_values = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5]
decay_values = [0.8, 0.85, 0.9, 0.95, 0.99]
epsilon_values = [1e-5, 1e-10]

best_score = float('inf')
best_hyperparameters = None

for max_iterations in max_iterations_values:
    for learning_rate in learning_rate_values:
        for decay in decay_values:
            for epsilon in epsilon_values:
                lr_model_test = LogisticRegression(max_iterations, learning_rate, decay, epsilon)
                lr_model_test.fit(X_train, Y_train)

                # Evaluate the model on the validation set
                y_val_pred = lr_model_test.predict(X_val)
                score = lr_model_test.binary_cross_entropy(y_val_pred, Y_val)

                # Check if the current combination of hyperparameters achieved a better score
                if score < best_score:
                    best_score = score
                    best_hyperparameters = (max_iterations, learning_rate, decay, epsilon)

print(f"Best Score: {best_score}")
print(f"Best Hyperparameters: {best_hyperparameters}")

Stopping Train at Iteration No.78
Training Complete with Binary Cross Entropy Loss of 0.5566518540336988
Training Complete with Binary Cross Entropy Loss of 0.5563779914995787
Stopping Train at Iteration No.77
Training Complete with Binary Cross Entropy Loss of 0.5550160361295557
Training Complete with Binary Cross Entropy Loss of 0.5560016172397695
Stopping Train at Iteration No.77
Training Complete with Binary Cross Entropy Loss of 0.5527806910969416
Training Complete with Binary Cross Entropy Loss of 0.5532842271468729
Stopping Train at Iteration No.77
Training Complete with Binary Cross Entropy Loss of 0.5545112951630043
Training Complete with Binary Cross Entropy Loss of 0.5549212866012389
Stopping Train at Iteration No.78
Training Complete with Binary Cross Entropy Loss of 0.5565202658827072
Training Complete with Binary Cross Entropy Loss of 0.554279642981674
Stopping Train at Iteration No.67
Training Complete with Binary Cross Entropy Loss of 0.4556851466299655
Training Complet

## Train model with best hyperparameters

In [30]:
max_iterations, learning_rate, decay, epsilon = (max_iterations, learning_rate, decay, epsilon)

model = LogisticRegression(max_iterations, learning_rate, decay, epsilon)

In [31]:
model.fit(X_train, Y_train)

Stopping Train at Iteration No.168
Training Complete with Binary Cross Entropy Loss of 0.4333968262668356


In [32]:
Y_pred = model.predict(X_test)

### Save Results as CSV file

In [33]:
result = pd.DataFrame({'PassengerId': testPassengerIds, 'Survived': Y_pred})

result.to_csv('output.csv', index=False)